# Examples

## Conversion ase <> cfg files

In [1]:
from pathlib import Path
from src.utils import atoms2cfg, cfg2atoms # Call the code installed.
# from utils import atoms2cfg, cfg2atoms # development
from ase.io import read

In [2]:
current = Path().cwd()
outcar_path = current.parents[1] / "mlip-2/test/examples/00.convert_vasp_outcar/OUTCAR"
traj = read(outcar_path, index=":")
file = "./example/test.cfg"
for i,atoms in enumerate(traj):
    append = False if i==0 else True
    atoms2cfg(atoms, file=file, append=append)
    

FileNotFoundError: [Errno 2] No such file or directory: '/home/tomo/git/mtp4py/mlip-2/test/examples/00.convert_vasp_outcar/OUTCAR'

In [ ]:
a = list(cfg2atoms(file="./example/test.cfg"))
b = list(cfg2atoms(file="./example/executable.cfg", symbols={1: "Ag"}))

In [4]:
a

[Atoms(symbols='Ag32', pbc=True, cell=[[8.458526959, 1.656720733, 1.673168563], [1.573491987, 8.821430406, -1.494376014], [1.591543182, -1.495192313, 8.82509342]], calculator=SinglePointCalculator(...)),
 Atoms(symbols='Ag32', pbc=True, cell=[[8.464367542, 1.654782691, 1.677485389], [1.570870501, 8.830852271, -1.499033421], [1.596387395, -1.500887298, 8.836299561]], calculator=SinglePointCalculator(...)),
 Atoms(symbols='Ag32', pbc=True, cell=[[8.466931764, 1.653931823, 1.679380628], [1.569719577, 8.834988801, -1.501078187], [1.598514175, -1.503387596, 8.841219451]], calculator=SinglePointCalculator(...))]

In [5]:
b

[Atoms(symbols='Ag32', pbc=True, cell=[[8.458527, 1.656721, 1.673169], [1.573492, 8.82143, -1.494376], [1.591543, -1.495192, 8.825093]], calculator=SinglePointCalculator(...)),
 Atoms(symbols='Ag32', pbc=True, cell=[[8.464368, 1.654783, 1.677485], [1.570871, 8.830852, -1.499033], [1.596387, -1.500887, 8.8363]], calculator=SinglePointCalculator(...)),
 Atoms(symbols='Ag32', pbc=True, cell=[[8.466932, 1.653932, 1.679381], [1.56972, 8.834989, -1.501078], [1.598514, -1.503388, 8.841219]], calculator=SinglePointCalculator(...))]

## Training

In [6]:
from src.training import MtpTraining # Call the code installed.
from pathlib import Path
# from training import MtpTraining # development

In [7]:
current = Path().cwd()
repo_path = current.parents[
    next(
        i
        for i in range(len(current.parents))
        if current.parents[i].name == "mtp4py"
    )
]
mtp_path = repo_path.parent / "mlip-2"

setting_path = mtp_path / "test/examples/01.train/06.mtp"
exe_path = mtp_path / "bin/mlp"
cfg_path = mtp_path / "test/examples/01.train/train.cfg"
dataset_name = cfg_path.name.split(".")[0]
output_path = repo_path / f"src/example/pot_{dataset_name}.almtp"

mtp = MtpTraining(setting_path, exe_path, cfg_path, output_path)
mtp.run()

/home/tomoyuki/git/mlip-2/bin/mlp train /home/tomoyuki/git/mlip-2/test/examples/01.train/06.mtp /home/tomoyuki/git/mlip-2/test/examples/01.train/train.cfg --trained-pot-name=/home/tomoyuki/git/mtp4py/src/example/pot_train.almtp --max-iter=100
MTPR from /home/tomoyuki/git/mlip-2/test/examples/01.train/06.mtp, Database: /home/tomoyuki/git/mlip-2/test/examples/01.train/train.cfg
Random initialization of radial coefficients
Rescaling...
   scaling = 0.833333333333333, condition number = 848.958997693705
   scaling = 0.909090909090909, condition number = 926.136632535035
   scaling = 1, condition number = 1018.74930264829
   scaling = 1.1, condition number = 1120.62333271922
   scaling = 1.2, condition number = 1222.49743610979
Rescaling to 0.833333333333333... done
Rescaling...
   scaling = 0.694444444444445, condition number = 707.468343705292
   scaling = 0.757575757575758, condition number = 771.782445907155
   scaling = 0.833333333333333, condition number = 848.959489340706
   scaling 

## Infer

In [8]:
from src.training import MtpTraining # Call the code installed.
# from ase_calculator_mtp import AseCalculatorMtp # development
from ase.optimize import FIRE


In [9]:
calc = AseCalculatorMtp(output_path)
cfgs = list(cfg2atoms(cfg_path))
atoms = cfgs[0].copy()
atoms.calc = calc
with FIRE(atoms, "./example/test_traj.traj") as fire:
    fire.run()


      Step     Time          Energy          fmax
FIRE:    0 16:51:01      -37.125699        0.479022
FIRE:    1 16:51:01      -37.140107        0.466766
FIRE:    2 16:51:01      -37.167052        0.441238
FIRE:    3 16:51:01      -37.202974        0.401957
FIRE:    4 16:51:01      -37.243016        0.347998
FIRE:    5 16:51:01      -37.281516        0.278577
FIRE:    6 16:51:01      -37.312666        0.194328
FIRE:    7 16:51:01      -37.330819        0.104855
FIRE:    8 16:51:01      -37.332275        0.108342
FIRE:    9 16:51:01      -37.332513        0.106365
FIRE:   10 16:51:01      -37.332974        0.102457
FIRE:   11 16:51:01      -37.333626        0.096707
FIRE:   12 16:51:01      -37.334424        0.089248
FIRE:   13 16:51:01      -37.335315        0.080257
FIRE:   14 16:51:01      -37.336240        0.069951
FIRE:   15 16:51:01      -37.337140        0.058597
FIRE:   16 16:51:01      -37.338040        0.046247


# ase db

In [47]:
from pathlib import Path
from ase.io import read
from ase.build import sort
from ase.calculators.emt import EMT
from ase.optimize.fire import FIRE
import numpy as np
from ase.db import connect

Path('test_ase_db.db').unlink
db = connect('test_ase_db.db')

def save_to_db(atoms):
    # 構造とエネルギーをデータベースに保存
    fmax = np.max(np.linalg.norm(atoms.get_forces(), axis=1))
    db.write(atoms, data={'fmax': fmax})

def opt(NiPt):
    NiPt = sort(NiPt)
    NiPt.positions += np.random.normal(0, 0.1, 3*len(NiPt)).reshape(-1, 3)

    NiPt.calc = EMT()
    opt = FIRE(NiPt)
    opt.attach(save_to_db, interval=1, atoms=NiPt)
    opt.run(fmax=0.05)

FePt = read("./FePt.cif")
FePt = sort(FePt)

NiPt = FePt.repeat([2,2,2])
NiPt.symbols[NiPt.symbols == "Fe"] = "Ni"
opt(NiPt)

NiPt = FePt.repeat([1,2,2])
NiPt.symbols[NiPt.symbols == "Fe"] = "Ni"
opt(NiPt)

NiPt = FePt.repeat([2,2,2])
ids = np.random.choice(np.where(NiPt.symbols == "Pt")[0], 3)
NiPt.symbols[ids] == "Ni"
NiPt.symbols[NiPt.symbols == "Fe"] = "Ni"
opt(NiPt)

NiPt = FePt.repeat([2,2,2])
NiPt.symbols[NiPt.symbols == "Fe"] = "Ni"
ids = np.random.choice(np.where(NiPt.symbols == "Ni")[0], 3)
NiPt.symbols[ids] == "Pt"
print(NiPt)
opt(NiPt)


      Step     Time          Energy          fmax
FIRE:    0 02:26:18        0.417211        2.431791


FIRE:    1 02:26:18        0.248524        1.930481
FIRE:    2 02:26:18        0.007299        1.196817
FIRE:    3 02:26:18       -0.205649        0.638960
FIRE:    4 02:26:18       -0.340130        0.541025
FIRE:    5 02:26:18       -0.393007        0.591091
FIRE:    6 02:26:18       -0.392332        0.834606
FIRE:    7 02:26:18       -0.396728        0.809058
FIRE:    8 02:26:18       -0.405009        0.759680
FIRE:    9 02:26:19       -0.416235        0.689599
FIRE:   10 02:26:19       -0.429185        0.602821
FIRE:   11 02:26:19       -0.442528        0.503613
FIRE:   12 02:26:19       -0.454993        0.395988
FIRE:   13 02:26:19       -0.465501        0.283437
FIRE:   14 02:26:19       -0.473911        0.157116
FIRE:   15 02:26:19       -0.478786        0.104761
FIRE:   16 02:26:19       -0.479508        0.182645
FIRE:   17 02:26:19       -0.479636        0.180921
FIRE:   18 02:26:19       -0.479887        0.177493
FIRE:   19 02:26:19       -0.480254        0.172406
FIRE:   20 0

In [72]:
# fmaxが0.1以下または1以上のエントリを取得
for row in db.select("fmax<0.1,fmax>0.1"):
    print(f"ID: {row.id}, fmax: {row.fmax}")

array([[-1.0653924 ,  1.29710299,  0.33264168],
       [-1.64056653, -0.05855819,  0.22296582],
       [ 0.46920201,  1.08480575,  0.33270722],
       [-1.3192419 , -0.28613262,  0.77333387],
       [ 0.73378549,  0.03663047,  2.38402479],
       [ 0.96503146, -0.30971816, -0.52702235],
       [ 1.953327  ,  0.48990843, -0.44648378],
       [ 1.88620724, -1.26837475, -0.3148493 ],
       [-0.95696508,  0.11151912, -1.12067912],
       [-0.37980359, -0.34251662, -0.19365004],
       [-1.1738205 , -0.17997014,  1.11923054],
       [-0.85073925, -0.62754532, -1.68368643],
       [ 0.23742927, -0.52410633, -0.42370019],
       [-0.8695219 ,  1.25285386, -1.27283277],
       [ 1.15599645, -0.44019055, -0.89892657],
       [ 0.96693389, -0.41607147,  0.39420779],
       [ 1.78431644, -1.19883126, -2.060609  ],
       [ 0.4294299 ,  0.22128911,  1.94947054],
       [-0.78639172, -0.99932712, -1.41342698],
       [ 0.43846084, -1.43680218,  0.46286562],
       [-0.84848067,  0.18802778, -0.412